# Sparsity & Intermittency Analysis

**Goal**: Measure the "Zero Problem" (frequency of no sales) and analyze sales time gaps.

> **Note**: Ensure this notebook is run in a configured Spark environment.

In [ ]:
import os
script_dir = os.getcwd()
FOLDER_PATH_DATA = os.path.join(script_dir, "..", "ETL Process", "final_optimized.parquet")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, lag
from pyspark.sql.window import Window

# Setup Spark Session
spark = SparkSession.builder \
    .appName("GapAnalysis") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Display Environment Setup (Optional)
pd.set_option('display.max_columns', None)

print("Libraries Loaded & Spark Session Started")

In [ ]:
# Load Data
try:
    # Use Spark to read large Parquet files
    df = spark.read.parquet(FOLDER_PATH_DATA)
    print("Data loaded successfully into Spark DataFrame")
    df.printSchema()
except Exception as e:
    print(f"Error loading data: {e}")

## Analysis A: Zero-Inflation Rate

**Strategy**:
1. **Total Count**: Calculate total number of rows.
2. **Zero Count**: Filter `sales == 0` and count.
3. **Visualization**: Plot a Pie Chart.

In [ ]:
# 1. Total Count
total_count = df.count()

# 2. Zero Count
zero_count = df.filter(col("sales") == 0).count()

# Calculation
sparsity_pct = (zero_count / total_count) * 100
non_zero_pct = 100 - sparsity_pct

print(f"Total Rows: {total_count}")
print(f"Zero Sales Rows: {zero_count}")
print(f"Sparsity Percentage: {sparsity_pct:.2f}%")

# 3. Visualization (Pie Chart)
labels = ['No Sales (Zeros)', 'Sales (>0)']
sizes = [sparsity_pct, non_zero_pct]
colors = ['#ff9999', '#66b3ff']
explode = (0.1, 0)

plt.figure(figsize=(8, 8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title(f"Zero-Inflation Rate (Sparsity: {sparsity_pct:.1f}%)")
plt.show()

## Analysis B: Gap Size Analysis

**Strategy**:
1. **Focus**: Select a slow-moving product (e.g., from HOBBIES).
2. **Fetch History**: Pull sorted daily sales.
3. **Calculate Gaps**: Days between non-zero sales.
4. **Visualization**: Histogram of gap sizes.

In [ ]:
# 1. Select a slow-moving item (e.g., first item from HOBBIES category)
# Ensure correct columns exist in df, here assuming: cat_id, item_id, date, sales

# Use Spark filter and select
target_item_row = df.filter(col("cat_id") == "HOBBIES").select("item_id").first()

if target_item_row:
    target_item_id = target_item_row[0]
    print(f"Analyzing Item: {target_item_id}")

    # Fetch dates where sales occurred for this item
    # Use date if available, or d if it is the only one
    # Here we assume column 'date' exists as date type or convertible string
    
    # Filter sales > 0
    # Use 'd' column (e.g., 'd_1') -> extract integer for sorting and calculation
    item_sales_df = df.filter((col("item_id") == target_item_id) & (col("sales") > 0)) \
                      .selectExpr("cast(substring(d, 3) as int) as day_num") \
                      .orderBy("day_num")
    
    # Convert to Pandas for simple calculations (data will be small after filtering)
    pdf_sales = item_sales_df.toPandas()
    
    if not pdf_sales.empty:
        # Calculate gaps (difference in days between each sale and the next)
        # diff() calculates difference between current and previous row
        pdf_sales['gap'] = pdf_sales['day_num'].diff()
        
        # Drop first NaN value
        gaps = pdf_sales['gap'].dropna()
        
        # 4. Visualization (Histogram)
        plt.figure(figsize=(10, 6))
        sns.histplot(gaps, bins=30, kde=False, color='orange')
        plt.title(f"Inter-arrival Time (Gap Size) for {target_item_id}")
        plt.xlabel("Gap Size (Days)")
        plt.ylabel("Frequency")
        plt.show()
        
        print(f"Average Gap Size: {gaps.mean():.2f} days")
        print(f"Max Gap Size: {gaps.max()} days")
    else:
        print("No sales found for this item.")
else:
    print("No HOBBIES items found or category column name mismatch.")